## WMLMIA 2018
## Tutorial 2 : DICOM

In [ ]:
import time
import scipy.io as sio
import numpy as np
from PIL import Image
from myFcnCart2Pol import myFcnCart2Pol
from skimage.segmentation import random_walker
import skimage
import scipy.signal as signal
import scipy.ndimage.morphology as morph
from myFcnPol2Cart import myFcnPol2Cart
import pickle
import os
import dicom

In [ ]:
imagePath = 'data/day1/tut01/Originaltotal/'
gtPath = 'data/day1/tut01/GTtotal/'

beta = 2000

# Size of image in polar coordinates
szPol = 256

# Window size for feature extraction
Kernelmat = np.array([3,5,7,13,17,21,25,30])

In [ ]:
# Loading trained Ranfom-Forest model for segmentation
pkl_file = open('data/day1/tut01/myfile.pkl','rb')
rf = pickle.load(pkl_file,encoding='latin1')
pkl_file.close()

In [ ]:
N = len(os.listdir(imagePath))
for fileIdx in range(1,N+1):
    start_time = time.time()      
    if fileIdx < 10:    
        filename = 'frame_01_000'+str(fileIdx)+'_003.png'
    else:
        filename = 'frame_01_00'+str(fileIdx)+'_003.png'
    print('Image: '+str(fileIdx)+'/'+str(N+1))
    OrgImage = np.asarray(Image.open(imagePath+filename).convert('L'));
    GtImage = np.asarray(Image.open(gtPath+filename).convert('L'));
    (szCart1, szCart2) = OrgImage.shape;
    
    # Creating volume tesnsor
    if fileIdx==1:
        Vol = np.empty((szCart1, szCart2, 3, N), dtype=np.uint8)        
    
    polarOrgImage = myFcnCart2Pol(OrgImage, szPol, szPol);
    polarGtImage = myFcnCart2Pol(GtImage, szPol, szPol);
    markers = np.zeros(polarOrgImage.shape, dtype=np.uint);   
    markers[0,:] = 2;
    markers[szPol-1,:] = 1;
    labels = random_walker(skimage.img_as_float(polarOrgImage), markers, beta, mode='bf', copy=True, return_full_prob=True);   
    for scaleCtr in range(0,len(Kernelmat)):
        hScale = np.ones((Kernelmat[scaleCtr], Kernelmat[0]));
        hScale = hScale/np.sum(hScale);
        ConvMean  = signal.convolve2d(polarOrgImage, hScale, mode='same');
        SqMean = signal.convolve2d(polarOrgImage**2, hScale, mode='same');
        ConvVer = SqMean - ConvMean**2;
        if scaleCtr == 0:
            MeanSet = np.reshape(ConvMean, (1,np.product(ConvMean.shape)))[0];
            VerSet = np.reshape(ConvVer, (1,np.product(ConvVer.shape)))[0];
            ConfSet = np.reshape(labels[1,:,:], (1,np.product(labels[1,:,:].shape)))[0];
        else:
            MeanSet = np.vstack((MeanSet,np.reshape(ConvMean, (1,np.product(ConvMean.shape)))[0]));
            VerSet = np.vstack((VerSet,np.reshape(ConvVer, (1,np.product(ConvVer.shape)))[0]));
    TestingFeatureSet = np.zeros((np.product(polarOrgImage.shape),17));
    MeanSet = MeanSet.transpose();
    VerSet = VerSet.transpose();
    TestingFeatureSet[:,0:16] = np.hstack((MeanSet,VerSet));
    TestingFeatureSet[:,16] = ConfSet;
    Pred = rf.predict(TestingFeatureSet[:,0:17]);
    PredImage = np.reshape(Pred,(szPol,szPol));
    
    
    PredBwLum = np.zeros(PredImage.shape);     
    PredBwLum[PredImage == 1] = 1;
    PredBwMed = np.zeros(PredImage.shape);     
    PredBwMed[(PredImage ==1) | (PredImage ==2)] = 1;
    
    LumBw = skimage.morphology.dilation((skimage.morphology.erosion(PredBwLum, skimage.morphology.disk(2))), skimage.morphology.disk(2))
    LumBw = skimage.morphology.label(LumBw); 
    MedBw = skimage.morphology.dilation((skimage.morphology.erosion(PredBwMed, skimage.morphology.disk(2))), skimage.morphology.disk(2))
    MedBw = skimage.morphology.label(MedBw);
    LumBw[np.nonzero(LumBw >1)] = 0
    MedBw[np.nonzero(MedBw >1)] = 0
    LumBw = morph.binary_fill_holes(LumBw)
    MedBw = morph.binary_fill_holes(MedBw)
    
    Lum = myFcnPol2Cart(LumBw, szCart1, szCart2);
    Med = myFcnPol2Cart(MedBw, szCart1, szCart2);
    k = 15
    
    Lum = skimage.morphology.erosion((skimage.morphology.dilation(Lum, skimage.morphology.disk(k))), skimage.morphology.disk(k))
    Lum = skimage.morphology.label(Lum); 
    Lum[np.nonzero(Lum >1)] = 0
    Lum = morph.binary_fill_holes(Lum)
    Med = skimage.morphology.erosion((skimage.morphology.dilation(Med, skimage.morphology.disk(k))), skimage.morphology.disk(k))
    Med = morph.binary_fill_holes(Med)
    
    Media = Med ^ Lum
    Org = np.empty((szCart1, szCart2, 3), dtype=np.uint8)
    Org[:, :, 0] = Org[:, :, 1] = Org[:, :, 2] = OrgImage
    Org[Media == 1,0] = 255; Org[Media == 1,1] = 0; Org[Media == 1,2] = 0;
    Vol[:,:,:,fileIdx-1] = Org    
    print("Time taken for the execution of the code:  %s seconds" % (time.time() - start_time))

In [ ]:
A = dicom.read_file('data/day1/tut02/testVolume.dcm')  # loading a dummy .dcm file for meta data information

Vol = Vol.transpose(3,0,1,2)
A.PixelData = np.uint8(Vol).tostring()
A.NumberofFrames = str(np.shape(Vol)[0])
A.Rows = np.shape(Vol)[1]
A.Columns = np.shape(Vol)[2]
A.SamplesperPixel = 3
A.PhotometricInterpretation = 'RGB'
# Saving volume as dicom file
A.save_as('data/day1/tut02/IVUSVolume.dcm') 